# Test

In [1]:
%load_ext autoreload
%autoreload 2

### Import datasets

Import datasets using functions from src/data/data.py. Datasets are downloaded from huggingface and stored in /data. Once downloaded, datasets are loaded locally.

Run ```pip install -e .``` if module importing isn't working.

In [2]:
from src.data.data import get_in_domain, get_out_domain

in_domain_train, in_domain_test = get_in_domain()
out_domain = get_out_domain()

print(f"In domain:\n{in_domain_train}")
print(in_domain_train[1])

print(f"Out of domain:\n{out_domain}")
print(out_domain[10])

In domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 261802
})
{'premise': 'One of our number will carry out your instructions minutely.', 'hypothesis': 'A member of my team will execute your orders with immense precision.', 'label': 0, 'idx': 2}
Out of domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
    num_rows: 10000
})
{'premise': 'The president avoided the athlete .', 'hypothesis': 'The athlete avoided the president .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN president)) (VP (VBD avoided) (NP (DT the) (NN athlete))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN athlete)) (VP (VBD avoided) (NP (DT the) (NN president))) (. .)))', 'binary_parse_premise': '( ( The president ) ( ( avoided ( the athlete ) ) . ) )', 'binary_parse_hypothesis': '( ( The athlete ) ( ( avoided ( the pr

### Import models

Import models using methods from src/models/opt.py. Models are downloaded from huggingface and stored in /models/pretrained. Once downloaded, models are loaded locally.

In [3]:
from src.model.model import get_model
from src.utils import print_gpu_memory_usage

# Get SequenceClassification models
# model_opt125, tokenizer_opt125 = get_model(model_name='opt-125m', model_type='SequenceClassification', pretrained=True)
# model_opt350, tokenizer_opt350 = get_model(model_name='opt-350m', model_type='SequenceClassification', pretrained=True)

# Get CasualLM models
print_gpu_memory_usage()
model_opt125_causal, tokenizer_opt125_causal = get_model(model_name='opt-125m', model_type='CausalLM', pretrained=True)
print_gpu_memory_usage()
model_opt350_causal, tokenizer_opt350_causal = get_model(model_name='opt-350m', model_type='CausalLM', pretrained=True)
print_gpu_memory_usage()

Device: cuda
Memory Allocated: 0.00 GB
Memory Reserved: 0.00 GB
Device: cuda
Memory Allocated: 0.47 GB
Memory Reserved: 0.52 GB
Device: cuda
Memory Allocated: 1.70 GB
Memory Reserved: 1.74 GB


### Generate method
The generate method is a low level way to generate text using CausalLM models. We constrain the model to only respond with Yes or No using a DisjuntiveConstraint.

In [4]:
import torch
import numpy as np
from transformers.generation.beam_constraints import DisjunctiveConstraint
from src.finetuners.utils import apply_minimal_pattern, tokenize_dataset

random_idx = np.random.choice(range(len(in_domain_test)), 1)
eval_dataset_in = in_domain_test.select(random_idx)

# Verbalize and tokenize
# tokenizer_opt125_causal.padding_side = 'left'
eval_dataset_in = apply_minimal_pattern(eval_dataset_in)
tokenized_eval_dataset_in = tokenize_dataset(eval_dataset_in, tokenizer_opt125_causal, max_length=512)

print(f"Minimal pattern applied:\n{eval_dataset_in[0]}")
print(f"Tokenized:\n{tokenized_eval_dataset_in[0]}")

input_ids = tokenized_eval_dataset_in['input_ids'][0]
input_ids = torch.tensor(input_ids, device=model_opt125_causal.device).unsqueeze(0)    # Convert to PyTorch tensor

# Yes/No constraint
yes_token_id = tokenizer_opt125_causal.encode("Yes", add_special_tokens=False)
no_token_id = tokenizer_opt125_causal.encode("No", add_special_tokens=False)
force_words_ids = [yes_token_id, no_token_id]
constraint = DisjunctiveConstraint(nested_token_ids=force_words_ids)

# Generate model output
gen_tokens = model_opt125_causal.generate(
    input_ids,
    max_new_tokens=1,
    # temperature=0.5,
    constraints=[constraint],
    num_beams=2
)

# Decode the generated tokens
generated_text = tokenizer_opt125_causal.batch_decode(gen_tokens[:, input_ids.shape[1]:], skip_special_tokens=True, clean_up_tokenization_spaces=False)

print(f"Generated text:\n{generated_text}")
print(f"Actual label: {eval_dataset_in['label']}")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Minimal pattern applied:
{'premise': 'As he emerged, Boris remarked, glancing up at the clock: "You are early.', 'hypothesis': "When he was appearing, Boris looked at the clock and said, 'You are early.'", 'label': 0, 'idx': 1236, 'text': 'As he emerged, Boris remarked, glancing up at the clock: "You are early. When he was appearing, Boris looked at the clock and said, \'You are early.\'?'}
Tokenized:
{'premise': 'As he emerged, Boris remarked, glancing up at the clock: "You are early.', 'hypothesis': "When he was appearing, Boris looked at the clock and said, 'You are early.'", 'label': 0, 'idx': 1236, 'text': 'As he emerged, Boris remarked, glancing up at the clock: "You are early. When he was appearing, Boris looked at the clock and said, \'You are early.\'?', 'input_ids': [2, 1620, 37, 4373, 6, 14335, 18767, 6, 5921, 7710, 62, 23, 5, 6700, 35, 22, 1185, 32, 419, 4, 520, 37, 21, 8165, 6, 14335, 1415, 23, 5, 6700, 8, 26, 6, 128, 1185, 32, 419, 955, 116, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### In-Context Learning
We use Seq2SeqTrainer to evaluate our in and out of domain sets with context

In [5]:
from src.finetuners.incontext import evaluate
import json

# Generate training and evaluation datasets. These should be used for all fine-tuning methods to ensure consistency. np.random should be seeded before this.
random_idxs = np.random.choice(range(len(in_domain_test)), 8)
# random_idxs = [3827]
eval_dataset_in = in_domain_test.select(random_idxs)
eval_dataset_out = out_domain.select(random_idxs)
# print(f"Using random dataset indices: {random_idxs.tolist()}")

combined_metrics = evaluate(model=model_opt125_causal, 
                            tokenizer=tokenizer_opt125_causal, 
                            eval_dataset_in=eval_dataset_in, 
                            eval_dataset_out=eval_dataset_out,
                            context='',
                            batch_size=8,
                            verbose=True, 
                            disable_tqdm=False)

print(f"Metrics:\n{json.dumps(combined_metrics, indent=4)}")

  0%|          | 0/1 [00:00<?, ?it/s]

In domain eval metrics:
{'loss': 0.9082330241799355, 'accuracy': 0.25, 'runtime': 0.7421000003814697, 'samples_per_second': 10.780218293879091}


  0%|          | 0/1 [00:00<?, ?it/s]

Out of domain eval metrics:
{'loss': 0.5332330353558064, 'accuracy': 0.875, 'runtime': 0.6362981796264648, 'samples_per_second': 12.572721808973826}
Metrics:
{
    "eval_in_loss": 0.9082330241799355,
    "eval_in_accuracy": 0.25,
    "eval_in_runtime": 0.7421000003814697,
    "eval_in_samples_per_second": 10.780218293879091,
    "eval_out_loss": 0.5332330353558064,
    "eval_out_accuracy": 0.875,
    "eval_out_runtime": 0.6362981796264648,
    "eval_out_samples_per_second": 12.572721808973826
}


### Batch ICL evaluation

In [6]:
from src.finetuners.incontext import batch_evaluate
from src.data.data import get_random_subsets
import json

# Generate training and evaluation datasets. These should be used for all fine-tuning methods to ensure consistency. np.random should be seeded before this.
train_datasets, eval_dataset_in, eval_dataset_out = get_random_subsets(train_dataset=in_domain_train, 
                                                                       eval_dataset_in=in_domain_test, 
                                                                       eval_dataset_out=out_domain, 
                                                                       train_sample_sizes=[2, 4],#[2, 16, 32, 64, 128], # [2, 4],
                                                                       num_trials=2,   # 5
                                                                       eval_sample_size=50) # 10

batch_evaluate(model_names=['opt-125m'], train_datasets=train_datasets, eval_dataset_in=eval_dataset_in, eval_dataset_out=eval_dataset_out)

'''
for context_data in [train_data[0] for train_data in train_datasets.values()]:
    results = batch_evaluate(model_name='opt-125m', train_dataset=context_data, eval_dataset_in=eval_dataset_in, eval_dataset_out=eval_dataset_out)

    print(f"Metrics:\n{json.dumps(results, indent=4)}")
'''

TypeError: batch_evaluate() got an unexpected keyword argument 'model_name'